# Глубокие нейронные сети

В этой тетрадке мы рассмотрим:
- Архитектуру глубоких нейронных сетей
- Проблему затухающего и взрывающегося градиента
- Методы борьбы с переобучением
- Современные архитектуры (ResNet)
- Практические примеры и визуализации

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification, load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

## 1. Архитектура глубоких нейронных сетей

### Что такое глубокая нейронная сеть?

**Глубокая нейронная сеть (DNN)** — это нейронная сеть с несколькими скрытыми слоями.

```
Входной слой → Скрытый слой 1 → Скрытый слой 2 → ... → Скрытый слой N → Выходной слой
```

### Почему глубина важна?

- **Иерархическое представление**: Каждый слой извлекает признаки разного уровня абстракции
- **Композициональность**: Глубокие сети могут представлять сложные функции через композицию простых
- **Эффективность**: Глубокие сети могут быть экспоненциально эффективнее мелких для некоторых задач

### Основные компоненты:

1. **Слои (Layers)**: Fully connected, convolutional, recurrent, etc.
2. **Функции активации**: ReLU, Sigmoid, Tanh, Leaky ReLU, ELU, etc.
3. **Инициализация весов**: Xavier, He, etc.
4. **Оптимизаторы**: SGD, Adam, RMSprop, etc.
5. **Регуляризация**: Dropout, L2, Batch Normalization, etc.

## 2. Функции активации

Функции активации добавляют нелинейность в сеть.

In [ ]:
class Activations:
    """Различные функции активации и их производные"""
    
    @staticmethod
    def sigmoid(z):
        return 1 / (1 + np.exp(-np.clip(z, -500, 500)))
    
    @staticmethod
    def sigmoid_derivative(z):
        s = Activations.sigmoid(z)
        return s * (1 - s)
    
    @staticmethod
    def tanh(z):
        return np.tanh(z)
    
    @staticmethod
    def tanh_derivative(z):
        return 1 - np.tanh(z) ** 2
    
    @staticmethod
    def relu(z):
        return np.maximum(0, z)
    
    @staticmethod
    def relu_derivative(z):
        return (z > 0).astype(float)
    
    @staticmethod
    def leaky_relu(z, alpha=0.01):
        return np.where(z > 0, z, alpha * z)
    
    @staticmethod
    def leaky_relu_derivative(z, alpha=0.01):
        return np.where(z > 0, 1, alpha)
    
    @staticmethod
    def elu(z, alpha=1.0):
        return np.where(z > 0, z, alpha * (np.exp(z) - 1))
    
    @staticmethod
    def elu_derivative(z, alpha=1.0):
        return np.where(z > 0, 1, alpha * np.exp(z))


# Визуализация функций активации
x = np.linspace(-5, 5, 1000)

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Функции активации и их производные', fontsize=16)

# Sigmoid
axes[0, 0].plot(x, Activations.sigmoid(x), 'b-', linewidth=2)
axes[0, 0].set_title('Sigmoid')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].set_ylabel('Активация')
axes[1, 0].plot(x, Activations.sigmoid_derivative(x), 'r-', linewidth=2)
axes[1, 0].set_title('Производная Sigmoid')
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].set_ylabel('Градиент')

# Tanh
axes[0, 1].plot(x, Activations.tanh(x), 'b-', linewidth=2)
axes[0, 1].set_title('Tanh')
axes[0, 1].grid(True, alpha=0.3)
axes[1, 1].plot(x, Activations.tanh_derivative(x), 'r-', linewidth=2)
axes[1, 1].set_title('Производная Tanh')
axes[1, 1].grid(True, alpha=0.3)

# ReLU
axes[0, 2].plot(x, Activations.relu(x), 'b-', linewidth=2)
axes[0, 2].set_title('ReLU')
axes[0, 2].grid(True, alpha=0.3)
axes[1, 2].plot(x, Activations.relu_derivative(x), 'r-', linewidth=2)
axes[1, 2].set_title('Производная ReLU')
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Сравнение продвинутых активаций
plt.figure(figsize=(15, 5))

plt.subplot(1, 2, 1)
plt.plot(x, Activations.relu(x), label='ReLU', linewidth=2)
plt.plot(x, Activations.leaky_relu(x), label='Leaky ReLU', linewidth=2)
plt.plot(x, Activations.elu(x), label='ELU', linewidth=2)
plt.title('Сравнение ReLU-подобных функций')
plt.xlabel('x')
plt.ylabel('f(x)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(x, Activations.relu_derivative(x), label='ReLU', linewidth=2)
plt.plot(x, Activations.leaky_relu_derivative(x), label='Leaky ReLU', linewidth=2)
plt.plot(x, Activations.elu_derivative(x), label='ELU', linewidth=2)
plt.title('Производные ReLU-подобных функций')
plt.xlabel('x')
plt.ylabel("f'(x)")
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Сравнение функций активации:

| Функция | Диапазон | Преимущества | Недостатки |
|---------|----------|--------------|------------|
| **Sigmoid** | (0, 1) | Гладкая, вероятностная интерпретация | Затухающий градиент, не центрирована |
| **Tanh** | (-1, 1) | Центрирована, гладкая | Затухающий градиент |
| **ReLU** | [0, ∞) | Быстрая, нет затухания для x>0 | "Dying ReLU" для x<0 |
| **Leaky ReLU** | (-∞, ∞) | Решает проблему "dying ReLU" | Требует настройки alpha |
| **ELU** | (-α, ∞) | Гладкая, робастная | Более медленная |

**Рекомендации:**
- Начинайте с **ReLU** — работает в большинстве случаев
- Для рекуррентных сетей попробуйте **Tanh**
- Если есть проблема "dying ReLU", используйте **Leaky ReLU** или **ELU**
- Для выходного слоя: **Sigmoid** (бинарная классификация), **Softmax** (многоклассовая)

## 3. Проблема затухающего градиента

### Что это?

В глубоких сетях градиенты могут становиться экспоненциально малыми при обратном распространении через множество слоев.

### Математика:

При обратном распространении градиент умножается на производные функций активации:

$$\frac{\partial L}{\partial w_1} = \frac{\partial L}{\partial a_n} \cdot \frac{\partial a_n}{\partial a_{n-1}} \cdot ... \cdot \frac{\partial a_2}{\partial a_1} \cdot \frac{\partial a_1}{\partial w_1}$$

Для sigmoid: $\sigma'(x) \leq 0.25$, поэтому градиент уменьшается в $0.25^n$ раз для n слоев!

### Демонстрация:

In [ ]:
def demonstrate_vanishing_gradient():
    """Демонстрация затухающего градиента"""
    
    n_layers = 20
    input_val = np.linspace(-3, 3, 100)
    
    # Для Sigmoid
    sigmoid_gradients = []
    for _ in range(n_layers):
        grad = Activations.sigmoid_derivative(input_val)
        sigmoid_gradients.append(np.mean(grad))
        input_val = Activations.sigmoid(input_val)
    
    # Для Tanh
    input_val = np.linspace(-3, 3, 100)
    tanh_gradients = []
    for _ in range(n_layers):
        grad = Activations.tanh_derivative(input_val)
        tanh_gradients.append(np.mean(grad))
        input_val = Activations.tanh(input_val)
    
    # Для ReLU
    input_val = np.linspace(-3, 3, 100)
    relu_gradients = []
    for _ in range(n_layers):
        grad = Activations.relu_derivative(input_val)
        relu_gradients.append(np.mean(grad))
        input_val = Activations.relu(input_val)
    
    # Визуализация
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    layers = np.arange(1, n_layers + 1)
    plt.plot(layers, sigmoid_gradients, 'o-', label='Sigmoid', linewidth=2)
    plt.plot(layers, tanh_gradients, 's-', label='Tanh', linewidth=2)
    plt.plot(layers, relu_gradients, '^-', label='ReLU', linewidth=2)
    plt.xlabel('Номер слоя')
    plt.ylabel('Средний градиент')
    plt.title('Затухание градиента по слоям')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.subplot(1, 2, 2)
    plt.semilogy(layers, sigmoid_gradients, 'o-', label='Sigmoid', linewidth=2)
    plt.semilogy(layers, tanh_gradients, 's-', label='Tanh', linewidth=2)
    plt.semilogy(layers, relu_gradients, '^-', label='ReLU', linewidth=2)
    plt.xlabel('Номер слоя')
    plt.ylabel('Средний градиент (log scale)')
    plt.title('Затухание градиента (логарифмическая шкала)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("Градиенты после 20 слоев:")
    print(f"Sigmoid: {sigmoid_gradients[-1]:.10f}")
    print(f"Tanh: {tanh_gradients[-1]:.10f}")
    print(f"ReLU: {relu_gradients[-1]:.6f}")

demonstrate_vanishing_gradient()

## 4. Проблема взрывающегося градиента

### Что это?

Противоположная проблема: градиенты становятся экспоненциально большими.

### Причины:
- Плохая инициализация весов (слишком большие значения)
- Высокая скорость обучения
- Неправильная архитектура

### Симптомы:
- Loss = NaN или Inf
- Веса становятся очень большими
- Нестабильное обучение

In [ ]:
def demonstrate_exploding_gradient():
    """Демонстрация взрывающегося градиента"""
    
    n_layers = 10
    input_size = 100
    
    # Плохая инициализация (большие веса)
    bad_gradients = []
    gradient = np.ones(input_size)
    for i in range(n_layers):
        weight = np.random.randn(input_size, input_size) * 2.0  # Большая дисперсия
        gradient = gradient @ weight
        bad_gradients.append(np.linalg.norm(gradient))
    
    # Правильная инициализация (Xavier)
    good_gradients = []
    gradient = np.ones(input_size)
    for i in range(n_layers):
        weight = np.random.randn(input_size, input_size) * np.sqrt(2.0 / input_size)
        gradient = gradient @ weight
        good_gradients.append(np.linalg.norm(gradient))
    
    # Визуализация
    plt.figure(figsize=(12, 5))
    
    layers = np.arange(1, n_layers + 1)
    
    plt.subplot(1, 2, 1)
    plt.plot(layers, bad_gradients, 'ro-', label='Плохая инициализация', linewidth=2)
    plt.plot(layers, good_gradients, 'go-', label='Xavier инициализация', linewidth=2)
    plt.xlabel('Номер слоя')
    plt.ylabel('Норма градиента')
    plt.title('Взрывающийся градиент')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.subplot(1, 2, 2)
    plt.semilogy(layers, bad_gradients, 'ro-', label='Плохая инициализация', linewidth=2)
    plt.semilogy(layers, good_gradients, 'go-', label='Xavier инициализация', linewidth=2)
    plt.xlabel('Номер слоя')
    plt.ylabel('Норма градиента (log scale)')
    plt.title('Взрывающийся градиент (логарифмическая шкала)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("Норма градиента после 10 слоев:")
    print(f"Плохая инициализация: {bad_gradients[-1]:.2e}")
    print(f"Xavier инициализация: {good_gradients[-1]:.2e}")

demonstrate_exploding_gradient()

## 5. Методы решения проблем градиентов

### 5.1. Правильная инициализация весов

**Xavier (Glorot) инициализация:**
$$W \sim \mathcal{N}\left(0, \sqrt{\frac{2}{n_{in} + n_{out}}}\right)$$

**He инициализация (для ReLU):**
$$W \sim \mathcal{N}\left(0, \sqrt{\frac{2}{n_{in}}}\right)$$

In [ ]:
class WeightInitializer:
    """Различные методы инициализации весов"""
    
    @staticmethod
    def zeros(shape):
        return np.zeros(shape)
    
    @staticmethod
    def random(shape, scale=0.01):
        return np.random.randn(*shape) * scale
    
    @staticmethod
    def xavier(shape):
        """Xavier (Glorot) инициализация"""
        fan_in, fan_out = shape[0], shape[1]
        limit = np.sqrt(6.0 / (fan_in + fan_out))
        return np.random.uniform(-limit, limit, shape)
    
    @staticmethod
    def he(shape):
        """He инициализация (для ReLU)"""
        fan_in = shape[0]
        std = np.sqrt(2.0 / fan_in)
        return np.random.randn(*shape) * std

### 5.2. Gradient Clipping

Ограничение нормы градиента:
$$\text{if } ||g|| > \text{threshold}: \quad g \leftarrow \frac{g}{||g||} \cdot \text{threshold}$$

In [ ]:
def gradient_clipping(gradients, max_norm=1.0):
    """Обрезка градиентов по норме"""
    total_norm = np.sqrt(sum(np.sum(g**2) for g in gradients))
    clip_coef = max_norm / (total_norm + 1e-6)
    if clip_coef < 1:
        return [g * clip_coef for g in gradients]
    return gradients

### 5.3. Batch Normalization

Нормализация входов каждого слоя:
$$\hat{x} = \frac{x - \mu_B}{\sqrt{\sigma_B^2 + \epsilon}}$$
$$y = \gamma \hat{x} + \beta$$

**Преимущества:**
- Ускоряет обучение
- Позволяет использовать большие скорости обучения
- Уменьшает чувствительность к инициализации
- Действует как регуляризатор

In [ ]:
class BatchNormalization:
    """Batch Normalization слой"""
    
    def __init__(self, num_features, epsilon=1e-5, momentum=0.9):
        self.epsilon = epsilon
        self.momentum = momentum
        self.gamma = np.ones((1, num_features))
        self.beta = np.zeros((1, num_features))
        self.running_mean = np.zeros((1, num_features))
        self.running_var = np.ones((1, num_features))
    
    def forward(self, X, training=True):
        if training:
            mean = np.mean(X, axis=0, keepdims=True)
            var = np.var(X, axis=0, keepdims=True)
            
            # Обновление скользящих средних
            self.running_mean = self.momentum * self.running_mean + (1 - self.momentum) * mean
            self.running_var = self.momentum * self.running_var + (1 - self.momentum) * var
            
            # Нормализация
            X_norm = (X - mean) / np.sqrt(var + self.epsilon)
        else:
            X_norm = (X - self.running_mean) / np.sqrt(self.running_var + self.epsilon)
        
        # Масштабирование и сдвиг
        out = self.gamma * X_norm + self.beta
        return out

## 6. Методы борьбы с переобучением

### Что такое переобучение?

Модель отлично работает на обучающих данных, но плохо на тестовых.

**Причины:**
- Слишком сложная модель
- Мало данных
- Слишком долгое обучение

### 6.1. L2-регуляризация (Ridge)

Добавляем штраф за большие веса:
$$L_{\text{total}} = L_{\text{data}} + \lambda \sum_i w_i^2$$

In [ ]:
def l2_regularization(weights, lambda_reg):
    """L2 регуляризация"""
    return lambda_reg * np.sum(weights ** 2)

### 6.2. Dropout

Случайное отключение нейронов во время обучения.

**Как работает:**
1. Во время обучения: каждый нейрон "выключается" с вероятностью p
2. Во время теста: все нейроны активны, выходы масштабируются

**Преимущества:**
- Предотвращает ко-адаптацию признаков
- Эквивалентно ансамблю множества сетей
- Очень эффективен против переобучения

In [ ]:
class Dropout:
    """Dropout слой"""
    
    def __init__(self, drop_prob=0.5):
        self.drop_prob = drop_prob
        self.mask = None
    
    def forward(self, X, training=True):
        if training:
            self.mask = np.random.binomial(1, 1 - self.drop_prob, X.shape) / (1 - self.drop_prob)
            return X * self.mask
        else:
            return X
    
    def backward(self, dout):
        return dout * self.mask


# Визуализация Dropout
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Исходная матрица
X = np.random.randn(10, 10)
axes[0].imshow(X, cmap='viridis', aspect='auto')
axes[0].set_title('Исходные активации')
axes[0].set_xlabel('Нейроны')
axes[0].set_ylabel('Примеры')

# С Dropout 0.5
dropout = Dropout(drop_prob=0.5)
X_drop = dropout.forward(X, training=True)
axes[1].imshow(X_drop, cmap='viridis', aspect='auto')
axes[1].set_title('С Dropout (p=0.5)')
axes[1].set_xlabel('Нейроны')

# Маска
axes[2].imshow(dropout.mask, cmap='gray', aspect='auto')
axes[2].set_title('Маска Dropout')
axes[2].set_xlabel('Нейроны')

plt.tight_layout()
plt.show()

### 6.3. Early Stopping

Остановка обучения при ухудшении качества на валидационной выборке.

### 6.4. Data Augmentation

Искусственное увеличение размера датасета (повороты, сдвиги, шум, и т.д.)

## 7. Реализация глубокой нейронной сети

In [ ]:
class DeepNeuralNetwork:
    """Глубокая нейронная сеть с продвинутыми техниками"""
    
    def __init__(self, layer_sizes, activation='relu', dropout_prob=0.0, use_batch_norm=False):
        self.layer_sizes = layer_sizes
        self.num_layers = len(layer_sizes) - 1
        self.activation = activation
        self.dropout_prob = dropout_prob
        self.use_batch_norm = use_batch_norm
        
        # Инициализация весов (He инициализация для ReLU)
        self.weights = []
        self.biases = []
        for i in range(self.num_layers):
            w = WeightInitializer.he((layer_sizes[i], layer_sizes[i+1]))
            b = np.zeros((1, layer_sizes[i+1]))
            self.weights.append(w)
            self.biases.append(b)
        
        # Batch Normalization слои
        if use_batch_norm:
            self.bn_layers = [BatchNormalization(size) for size in layer_sizes[1:]]
        
        # Dropout слои
        if dropout_prob > 0:
            self.dropout_layers = [Dropout(dropout_prob) for _ in range(self.num_layers - 1)]
        
        # История обучения
        self.train_losses = []
        self.val_losses = []
    
    def _activate(self, z):
        if self.activation == 'relu':
            return Activations.relu(z)
        elif self.activation == 'sigmoid':
            return Activations.sigmoid(z)
        elif self.activation == 'tanh':
            return Activations.tanh(z)
        else:
            return z
    
    def _activate_derivative(self, z):
        if self.activation == 'relu':
            return Activations.relu_derivative(z)
        elif self.activation == 'sigmoid':
            return Activations.sigmoid_derivative(z)
        elif self.activation == 'tanh':
            return Activations.tanh_derivative(z)
        else:
            return np.ones_like(z)
    
    def forward(self, X, training=True):
        """Прямое распространение"""
        self.activations = [X]
        self.zs = []
        
        A = X
        for i in range(self.num_layers):
            Z = np.dot(A, self.weights[i]) + self.biases[i]
            self.zs.append(Z)
            
            # Batch Normalization
            if self.use_batch_norm and i < self.num_layers - 1:
                Z = self.bn_layers[i].forward(Z, training)
            
            # Активация
            if i == self.num_layers - 1:  # Выходной слой (sigmoid для бинарной классификации)
                A = Activations.sigmoid(Z)
            else:
                A = self._activate(Z)
                
                # Dropout
                if self.dropout_prob > 0 and training:
                    A = self.dropout_layers[i].forward(A, training)
            
            self.activations.append(A)
        
        return A
    
    def backward(self, X, y, learning_rate, lambda_reg=0.01):
        """Обратное распространение с L2 регуляризацией"""
        m = X.shape[0]
        
        # Градиент выходного слоя
        dA = self.activations[-1] - y.reshape(-1, 1)
        
        # Обратное распространение через слои
        for i in reversed(range(self.num_layers)):
            dZ = dA * Activations.sigmoid_derivative(self.zs[i]) if i == self.num_layers - 1 else dA * self._activate_derivative(self.zs[i])
            
            # Градиенты весов и смещений
            dW = np.dot(self.activations[i].T, dZ) / m + (lambda_reg / m) * self.weights[i]
            db = np.sum(dZ, axis=0, keepdims=True) / m
            
            # Обновление параметров
            self.weights[i] -= learning_rate * dW
            self.biases[i] -= learning_rate * db
            
            # Градиент для предыдущего слоя
            if i > 0:
                dA = np.dot(dZ, self.weights[i].T)
                
                # Dropout градиент
                if self.dropout_prob > 0:
                    dA = self.dropout_layers[i-1].backward(dA)
    
    def train(self, X_train, y_train, X_val=None, y_val=None, epochs=1000, 
              learning_rate=0.01, lambda_reg=0.01, batch_size=32, verbose=True):
        """Обучение сети с mini-batch gradient descent"""
        
        n_samples = X_train.shape[0]
        
        for epoch in range(epochs):
            # Перемешивание данных
            indices = np.random.permutation(n_samples)
            X_shuffled = X_train[indices]
            y_shuffled = y_train[indices]
            
            # Mini-batch обучение
            for i in range(0, n_samples, batch_size):
                X_batch = X_shuffled[i:i+batch_size]
                y_batch = y_shuffled[i:i+batch_size]
                
                # Прямое и обратное распространение
                self.forward(X_batch, training=True)
                self.backward(X_batch, y_batch, learning_rate, lambda_reg)
            
            # Вычисление loss
            train_pred = self.forward(X_train, training=False)
            train_loss = self._compute_loss(y_train, train_pred, lambda_reg)
            self.train_losses.append(train_loss)
            
            if X_val is not None and y_val is not None:
                val_pred = self.forward(X_val, training=False)
                val_loss = self._compute_loss(y_val, val_pred, lambda_reg)
                self.val_losses.append(val_loss)
            
            if verbose and epoch % 100 == 0:
                if X_val is not None:
                    print(f'Epoch {epoch}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}')
                else:
                    print(f'Epoch {epoch}: Train Loss = {train_loss:.4f}')
    
    def _compute_loss(self, y, y_pred, lambda_reg):
        """Вычисление loss с L2 регуляризацией"""
        m = y.shape[0]
        y = y.reshape(-1, 1)
        
        # Binary cross-entropy
        bce_loss = -np.mean(y * np.log(y_pred + 1e-8) + (1 - y) * np.log(1 - y_pred + 1e-8))
        
        # L2 регуляризация
        l2_loss = (lambda_reg / (2 * m)) * sum(np.sum(w ** 2) for w in self.weights)
        
        return bce_loss + l2_loss
    
    def predict(self, X):
        """Предсказание классов"""
        y_pred = self.forward(X, training=False)
        return (y_pred > 0.5).astype(int).flatten()
    
    def predict_proba(self, X):
        """Предсказание вероятностей"""
        return self.forward(X, training=False).flatten()

## 8. Сравнение техник на практике

In [ ]:
# Генерация данных
X, y = make_classification(
    n_samples=1000,
    n_features=20,
    n_informative=15,
    n_redundant=5,
    n_classes=2,
    random_state=42
)

# Разделение на train/val/test
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)

# Нормализация
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

print(f"Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")

In [ ]:
# Эксперимент 1: Базовая сеть (без регуляризации)
print("=" * 50)
print("Эксперимент 1: Базовая сеть")
print("=" * 50)

dnn_basic = DeepNeuralNetwork(
    layer_sizes=[20, 64, 32, 16, 1],
    activation='relu',
    dropout_prob=0.0,
    use_batch_norm=False
)

dnn_basic.train(
    X_train, y_train, X_val, y_val,
    epochs=500,
    learning_rate=0.01,
    lambda_reg=0.0,
    batch_size=32,
    verbose=False
)

train_acc = np.mean(dnn_basic.predict(X_train) == y_train)
val_acc = np.mean(dnn_basic.predict(X_val) == y_val)
test_acc = np.mean(dnn_basic.predict(X_test) == y_test)

print(f"Train Accuracy: {train_acc:.4f}")
print(f"Val Accuracy: {val_acc:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")

In [ ]:
# Эксперимент 2: С Dropout
print("\n" + "=" * 50)
print("Эксперимент 2: С Dropout")
print("=" * 50)

dnn_dropout = DeepNeuralNetwork(
    layer_sizes=[20, 64, 32, 16, 1],
    activation='relu',
    dropout_prob=0.3,
    use_batch_norm=False
)

dnn_dropout.train(
    X_train, y_train, X_val, y_val,
    epochs=500,
    learning_rate=0.01,
    lambda_reg=0.0,
    batch_size=32,
    verbose=False
)

train_acc = np.mean(dnn_dropout.predict(X_train) == y_train)
val_acc = np.mean(dnn_dropout.predict(X_val) == y_val)
test_acc = np.mean(dnn_dropout.predict(X_test) == y_test)

print(f"Train Accuracy: {train_acc:.4f}")
print(f"Val Accuracy: {val_acc:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")

In [ ]:
# Эксперимент 3: С L2 регуляризацией
print("\n" + "=" * 50)
print("Эксперимент 3: С L2 регуляризацией")
print("=" * 50)

dnn_l2 = DeepNeuralNetwork(
    layer_sizes=[20, 64, 32, 16, 1],
    activation='relu',
    dropout_prob=0.0,
    use_batch_norm=False
)

dnn_l2.train(
    X_train, y_train, X_val, y_val,
    epochs=500,
    learning_rate=0.01,
    lambda_reg=0.01,
    batch_size=32,
    verbose=False
)

train_acc = np.mean(dnn_l2.predict(X_train) == y_train)
val_acc = np.mean(dnn_l2.predict(X_val) == y_val)
test_acc = np.mean(dnn_l2.predict(X_test) == y_test)

print(f"Train Accuracy: {train_acc:.4f}")
print(f"Val Accuracy: {val_acc:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")

In [ ]:
# Эксперимент 4: Все вместе (Dropout + L2 + BatchNorm)
print("\n" + "=" * 50)
print("Эксперимент 4: Dropout + L2 + BatchNorm")
print("=" * 50)

dnn_full = DeepNeuralNetwork(
    layer_sizes=[20, 64, 32, 16, 1],
    activation='relu',
    dropout_prob=0.3,
    use_batch_norm=True
)

dnn_full.train(
    X_train, y_train, X_val, y_val,
    epochs=500,
    learning_rate=0.01,
    lambda_reg=0.01,
    batch_size=32,
    verbose=False
)

train_acc = np.mean(dnn_full.predict(X_train) == y_train)
val_acc = np.mean(dnn_full.predict(X_val) == y_val)
test_acc = np.mean(dnn_full.predict(X_test) == y_test)

print(f"Train Accuracy: {train_acc:.4f}")
print(f"Val Accuracy: {val_acc:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")

In [ ]:
# Визуализация кривых обучения
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

models = [
    (dnn_basic, 'Базовая сеть'),
    (dnn_dropout, 'С Dropout'),
    (dnn_l2, 'С L2'),
    (dnn_full, 'Dropout + L2 + BN')
]

for idx, (model, title) in enumerate(models):
    ax = axes[idx // 2, idx % 2]
    ax.plot(model.train_losses, label='Train Loss', linewidth=2)
    ax.plot(model.val_losses, label='Val Loss', linewidth=2)
    ax.set_xlabel('Эпоха')
    ax.set_ylabel('Loss')
    ax.set_title(title)
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 9. Residual Networks (ResNet)

### Проблема очень глубоких сетей

Даже с ReLU и правильной инициализацией, очень глубокие сети (50+ слоев) трудно обучать.

### Идея ResNet

Вместо обучения функции $H(x)$, обучаем остаточную функцию:
$$F(x) = H(x) - x$$

Тогда:
$$H(x) = F(x) + x$$

Это называется **skip connection** или **residual connection**.

### Преимущества:
- Градиенты легко распространяются через skip connections
- Позволяет обучать сети с сотнями слоев
- Если слой не нужен, сеть может обучить F(x) ≈ 0

### Архитектура ResNet блока:

```
x → [Conv → BN → ReLU → Conv → BN] → (+) → ReLU → output
    ↓                                   ↑
    └──────────────────────────────────┘
              (skip connection)
```

In [ ]:
class ResidualBlock:
    """Residual блок для fully-connected сетей"""
    
    def __init__(self, input_size, hidden_size, dropout_prob=0.0):
        # Два слоя в residual блоке
        self.W1 = WeightInitializer.he((input_size, hidden_size))
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = WeightInitializer.he((hidden_size, input_size))
        self.b2 = np.zeros((1, input_size))
        
        self.bn1 = BatchNormalization(hidden_size)
        self.bn2 = BatchNormalization(input_size)
        
        if dropout_prob > 0:
            self.dropout = Dropout(dropout_prob)
        else:
            self.dropout = None
    
    def forward(self, X, training=True):
        """Прямое распространение с skip connection"""
        # Первый слой
        Z1 = np.dot(X, self.W1) + self.b1
        Z1 = self.bn1.forward(Z1, training)
        A1 = Activations.relu(Z1)
        
        if self.dropout is not None:
            A1 = self.dropout.forward(A1, training)
        
        # Второй слой
        Z2 = np.dot(A1, self.W2) + self.b2
        Z2 = self.bn2.forward(Z2, training)
        
        # Skip connection
        out = Activations.relu(Z2 + X)  # Residual: F(x) + x
        
        return out


class ResNet:
    """Residual Network"""
    
    def __init__(self, input_size, hidden_size, num_blocks, output_size, dropout_prob=0.0):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_blocks = num_blocks
        
        # Входной слой (проецирует в hidden_size)
        self.W_input = WeightInitializer.he((input_size, hidden_size))
        self.b_input = np.zeros((1, hidden_size))
        
        # Residual блоки
        self.blocks = [ResidualBlock(hidden_size, hidden_size, dropout_prob) 
                       for _ in range(num_blocks)]
        
        # Выходной слой
        self.W_output = WeightInitializer.he((hidden_size, output_size))
        self.b_output = np.zeros((1, output_size))
        
        self.train_losses = []
        self.val_losses = []
    
    def forward(self, X, training=True):
        """Прямое распространение"""
        # Входной слой
        A = np.dot(X, self.W_input) + self.b_input
        A = Activations.relu(A)
        
        # Residual блоки
        for block in self.blocks:
            A = block.forward(A, training)
        
        # Выходной слой
        Z = np.dot(A, self.W_output) + self.b_output
        out = Activations.sigmoid(Z)
        
        return out
    
    def predict(self, X):
        """Предсказание классов"""
        y_pred = self.forward(X, training=False)
        return (y_pred > 0.5).astype(int).flatten()
    
    def predict_proba(self, X):
        """Предсказание вероятностей"""
        return self.forward(X, training=False).flatten()

### Визуализация архитектуры ResNet

In [ ]:
# Визуализация разницы между обычной сетью и ResNet
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Обычная сеть
ax = axes[0]
ax.text(0.5, 0.9, 'Input', ha='center', va='center', fontsize=12, 
        bbox=dict(boxstyle='round', facecolor='lightblue'))
for i in range(5):
    y = 0.75 - i * 0.15
    ax.arrow(0.5, y + 0.05, 0, -0.08, head_width=0.05, head_length=0.02, fc='black', ec='black')
    ax.text(0.5, y, f'Layer {i+1}', ha='center', va='center', fontsize=11,
            bbox=dict(boxstyle='round', facecolor='lightcoral'))
ax.arrow(0.5, 0.05, 0, -0.08, head_width=0.05, head_length=0.02, fc='black', ec='black')
ax.text(0.5, -0.05, 'Output', ha='center', va='center', fontsize=12,
        bbox=dict(boxstyle='round', facecolor='lightgreen'))
ax.set_xlim(0, 1)
ax.set_ylim(-0.15, 1)
ax.axis('off')
ax.set_title('Обычная глубокая сеть', fontsize=14, fontweight='bold')

# ResNet
ax = axes[1]
ax.text(0.5, 0.9, 'Input', ha='center', va='center', fontsize=12,
        bbox=dict(boxstyle='round', facecolor='lightblue'))
for i in range(5):
    y = 0.75 - i * 0.15
    # Основной путь
    ax.arrow(0.5, y + 0.05, 0, -0.08, head_width=0.05, head_length=0.02, fc='black', ec='black')
    ax.text(0.5, y, f'Block {i+1}', ha='center', va='center', fontsize=11,
            bbox=dict(boxstyle='round', facecolor='lightcoral'))
    # Skip connection
    if i < 4:
        ax.annotate('', xy=(0.5, y - 0.05), xytext=(0.5, y + 0.05),
                   arrowprops=dict(arrowstyle='->', lw=2, color='blue',
                                 connectionstyle='arc3,rad=0.3'))
ax.arrow(0.5, 0.05, 0, -0.08, head_width=0.05, head_length=0.02, fc='black', ec='black')
ax.text(0.5, -0.05, 'Output', ha='center', va='center', fontsize=12,
        bbox=dict(boxstyle='round', facecolor='lightgreen'))
ax.text(0.75, 0.4, 'Skip\nConnections', ha='center', va='center', 
        fontsize=10, color='blue', fontweight='bold')
ax.set_xlim(0, 1)
ax.set_ylim(-0.15, 1)
ax.axis('off')
ax.set_title('Residual Network (ResNet)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

## 10. Ключевые выводы

### Проблемы глубоких сетей:

| Проблема | Причины | Решения |
|----------|---------|----------|
| **Затухающий градиент** | Sigmoid/Tanh активации, много слоев | ReLU, правильная инициализация, BatchNorm, ResNet |
| **Взрывающийся градиент** | Плохая инициализация, высокий LR | Gradient clipping, Xavier/He init, BatchNorm |
| **Переобучение** | Сложная модель, мало данных | Dropout, L2, Early stopping, Data augmentation |
| **Медленное обучение** | Плохая инициализация | BatchNorm, Adam optimizer |

### Лучшие практики:

1. **Функции активации:**
   - Скрытые слои: ReLU или его варианты
   - Выходной слой: Sigmoid (бинарная), Softmax (многоклассовая)

2. **Инициализация весов:**
   - Xavier для Sigmoid/Tanh
   - He для ReLU

3. **Регуляризация:**
   - Начните с Dropout (0.3-0.5)
   - Добавьте L2 если нужно
   - Используйте BatchNorm

4. **Архитектура:**
   - Для очень глубоких сетей (50+ слоев) используйте skip connections (ResNet)
   - Начинайте с простой архитектуры и усложняйте

5. **Обучение:**
   - Mini-batch gradient descent
   - Оптимизатор Adam
   - Learning rate scheduling
   - Early stopping

### ResNet революция:

- Позволил обучать сети с 100+ слоями
- Skip connections решают проблему затухающего градиента
- Стал основой для многих современных архитектур
- Идея residual learning применима к разным типам сетей

### Практические рекомендации:

1. Всегда нормализуйте входные данные
2. Используйте валидационную выборку для мониторинга
3. Визуализируйте кривые обучения
4. Экспериментируйте с гиперпараметрами
5. Начинайте с простых моделей
6. Используйте готовые библиотеки (PyTorch, TensorFlow) для продакшна

## 11. Задания для самостоятельной работы

1. **Градиенты:**
   - Реализуйте визуализацию градиентов по слоям во время обучения
   - Сравните затухание градиентов для разных функций активации

2. **Регуляризация:**
   - Реализуйте L1 регуляризацию
   - Попробуйте Elastic Net (L1 + L2)
   - Реализуйте Early Stopping

3. **Оптимизаторы:**
   - Реализуйте Adam optimizer
   - Сравните SGD, Momentum, RMSprop, Adam

4. **Архитектура:**
   - Реализуйте ResNet с несколькими residual блоками
   - Обучите на MNIST или CIFAR-10
   - Сравните с обычной глубокой сетью той же глубины

5. **Анализ:**
   - Визуализируйте веса сети
   - Постройте confusion matrix
   - Анализ ошибок модели

6. **Продвинутое:**
   - Реализуйте DenseNet (dense connections)
   - Попробуйте cyclical learning rates
   - Реализуйте mixup augmentation